In [1]:
import arcpy
arcpy.env.workspace = r'C:\Users\ltmsbi\Documents\ArcGIS\Projects\Final_Deployment\Final_Deployment.gdb'

In [4]:
# Create layers per-species and geometry then add names to list for spatial joining


# Create list of n + n*3 files to process, where n is the number of species
ps_records = [] # empty list to append items onto

# Create dictionary of long names
# Values are not used in this code and are only present for human readability
species_fullnames = {
    "phrag": "Phragmites, Unspecified",
    "knot": "Knotweed, Unspecified",
    "wp": "Wild Parsnip",
    "toh": "Tree-of-Heaven (Ailanthus)",
    "pl": "Purple Loosestrife"
}

# Extract only the keys to a list
species_shortnames = list(species_fullnames.keys())


# Add model data layer names to processing list ----------


# Define the threshold used to determine presences
threshold = 'precision'

# Add names to list
for n in range(0,len(species_shortnames)):
    # Add n items for model data 
    ps_records.append('SVI_Project_presences_'+species_shortnames[n]+"_"+threshold)

    
# ----------
    
# Separate out iMap records by species and geometry ----------


# IDs that iMap assigns to the various species of interest
# Used for building queries to select by attribute
jurisdiction_ids = {
    "phrag": 1277,
    "wp": 1182,
    "pl": 1265,
    "toh": 1167,
    "knot": (1074, 1191, 1278, 1479) # Includes Japanese knotweed, giant knotweed, bohemian knotweed, and knotweed species unknown
}

# Define list of geometries in the iMap data
imap_geometries = ["POINT", "LINE", "POLYGON"]

# Define imap records types
imap_record_types = {
    "cnfrm": "", # The "suffix" for confirmed records is blank
    "uncnfrm": "_UNCONFIRMED"
}

# Add n*2*3 + n items for iMap data (accounts for 2 record and 3 geometry types, 
# plus not-detected records) 
for n in range(0,len(species_shortnames)):
    # Set up per-species query for selecting by attribute
    if species_shortnames[n] is "knot":
        # Set initial SQL query
        idClause = "jurisdiction_species_id = "+str(jurisdiction_ids["knot"][0])
        # Add more conditions to query
        for ID in jurisdiction_ids["knot"][1:]:
            idClause = idClause + " Or jurisdiction_species_id = " + str(ID)
    else:
        idClause = "jurisdiction_species_id = "+str(jurisdiction_ids[species_shortnames[n]]) 
    
    # Copy a per-species subset of not detected polygons
    sel = arcpy.management.SelectLayerByAttribute("NOT_DETECTED_POLYGON", "NEW_SELECTION", idClause)
    imap_nd = "iMap_nd_"+species_shortnames[n]
    arcpy.management.CopyFeatures(sel, imap_nd)
    ps_records.append(imap_nd)
    # Remove variables and selections to save memory
    del sel, imap_nd
    arcpy.management.SelectLayerByAttribute("NOT_DETECTED_POLYGON", "CLEAR_SELECTION")
    
    # Copy a per-species subset for each record and geometry type
    for rt in range(0,len(imap_record_types)):
        for g in range(0,len(imap_geometries)):
            sel = arcpy.management.SelectLayerByAttribute("PRESENCE_"+imap_geometries[g]+list(imap_record_types.values())[rt], "NEW_SELECTION", idClause)
            imap_subset = "iMap_"+imap_geometries[g].lower()+"_"+list(imap_record_types.keys())[rt]+"_"+species_shortnames[n]
            arcpy.management.CopyFeatures(sel, imap_subset)
            ps_records.append(imap_subset)
            # Remove variables and selections to save memory
            del sel, imap_subset
            arcpy.management.SelectLayerByAttribute("PRESENCE_"+imap_geometries[g], "CLEAR_SELECTION")
        
        
# ----------

print("Number of layers to process: "+str(len(ps_records)))
print(ps_records)

Number of layers to process: 40
['SVI_Project_presences_phrag_precision', 'SVI_Project_presences_knot_precision', 'SVI_Project_presences_wp_precision', 'SVI_Project_presences_toh_precision', 'SVI_Project_presences_pl_precision', 'iMap_nd_phrag', 'iMap_point_cnfrm_phrag', 'iMap_line_cnfrm_phrag', 'iMap_polygon_cnfrm_phrag', 'iMap_point_uncnfrm_phrag', 'iMap_line_uncnfrm_phrag', 'iMap_polygon_uncnfrm_phrag', 'iMap_nd_knot', 'iMap_point_cnfrm_knot', 'iMap_line_cnfrm_knot', 'iMap_polygon_cnfrm_knot', 'iMap_point_uncnfrm_knot', 'iMap_line_uncnfrm_knot', 'iMap_polygon_uncnfrm_knot', 'iMap_nd_wp', 'iMap_point_cnfrm_wp', 'iMap_line_cnfrm_wp', 'iMap_polygon_cnfrm_wp', 'iMap_point_uncnfrm_wp', 'iMap_line_uncnfrm_wp', 'iMap_polygon_uncnfrm_wp', 'iMap_nd_toh', 'iMap_point_cnfrm_toh', 'iMap_line_cnfrm_toh', 'iMap_polygon_cnfrm_toh', 'iMap_point_uncnfrm_toh', 'iMap_line_uncnfrm_toh', 'iMap_polygon_uncnfrm_toh', 'iMap_nd_pl', 'iMap_point_cnfrm_pl', 'iMap_line_cnfrm_pl', 'iMap_polygon_cnfrm_pl', 'iMap

In [1]:
# The thresholdless code has these fields to keep
keepFields = keepFields + ['model_points', 'iMap_nd', 'OBJECTID', 'Shape', 'Shape_Area', 'Shape_Length']

ps_records = ['SVI_Project_presences_phrag_precision', 'SVI_Project_presences_knot_precision', 'SVI_Project_presences_wp_precision', 'SVI_Project_presences_toh_precision', 'SVI_Project_presences_pl_precision', 'iMap_nd_phrag', 'iMap_point_cnfrm_phrag', 'iMap_line_cnfrm_phrag', 'iMap_polygon_cnfrm_phrag', 'iMap_point_uncnfrm_phrag', 'iMap_line_uncnfrm_phrag', 'iMap_polygon_uncnfrm_phrag', 'iMap_nd_knot', 'iMap_point_cnfrm_knot', 'iMap_line_cnfrm_knot', 'iMap_polygon_cnfrm_knot', 'iMap_point_uncnfrm_knot', 'iMap_line_uncnfrm_knot', 'iMap_polygon_uncnfrm_knot', 'iMap_nd_wp', 'iMap_point_cnfrm_wp', 'iMap_line_cnfrm_wp', 'iMap_polygon_cnfrm_wp', 'iMap_point_uncnfrm_wp', 'iMap_line_uncnfrm_wp', 'iMap_polygon_uncnfrm_wp', 'iMap_nd_toh', 'iMap_point_cnfrm_toh', 'iMap_line_cnfrm_toh', 'iMap_polygon_cnfrm_toh', 'iMap_point_uncnfrm_toh', 'iMap_line_uncnfrm_toh', 'iMap_polygon_uncnfrm_toh', 'iMap_nd_pl', 'iMap_point_cnfrm_pl', 'iMap_line_cnfrm_pl', 'iMap_polygon_cnfrm_pl', 'iMap_point_uncnfrm_pl', 'iMap_line_uncnfrm_pl', 'iMap_polygon_uncnfrm_pl']
# Spatial join repeatedly

input_grid = "reporting_analysis_grid_thresholdless"

# Used for tracking iterations which enables field calculations after confirmed and unconfirmed iMap
# records of all geometries are spatially joined
counter = 1

# Effectively, for iMap and model data, each species, and geometry/record type (if iMap data):
for i, r in enumerate(ps_records):
    # Define field naming convention
    if "SVI" in r:
        # Slice to only retain the species and create new field name
        species = r[22:-10]
        field_name = "model_"+species
    else:
        field_name = r

    print(field_name)

    # Add fields for the per-species version that we want to keep
    keepFields.append(field_name)


    # Create field mappings for spatial join -----

    fieldMappings = arcpy.FieldMappings() #field mapping variable, this will store all field mappings

    # Create list of field names to keep in the output file
    mainTable = []
    for i in arcpy.ListFields(mainLayer):
        if i.name in (keepFields):
            mainTable.append(i.name)

    # List of input feature classes for the spatial join
    f = [input_grid, r]

    for k in mainTable: # loop through main table
        fieldMap = arcpy.FieldMap() # create an empty field map variable
        fieldMap.addInputField(mainLayer,k) # insert the target layer as the first input into the field map
        for feature in f: # loop through feature classes
            for field in arcpy.ListFields(feature): # loop through field of each feature class
                if k in field.name: # check if any field match with our target field then append it as an input field
                    fieldMap.addInputField(feature,field.name) 
        fieldMappings.addFieldMap(fieldMap) # add the current field map to the main field map variable

    # ----------

    # Perform the spatial join & clean the table----------

    out_grid = "tmp_grid_"+str(i)
    arcpy.analysis.SpatialJoin(input_grid,r,out_grid,"JOIN_ONE_TO_ONE","KEEP_ALL",fieldMappings)
    del fieldMappings, mainTable, f, fieldMap

    # Rename Join_Count field
    arcpy.management.AlterField(out_grid, "Join_Count", field_name)
    # Delete TARGET_FID field
    arcpy.management.DeleteField(out_grid, "TARGET_FID")

    #----------


    # Sum records only after confirmed or unconfirmed iMap records of all geometries are spatially joined ----------

    if i >= len(species_shortnames): # If our iteration index is past processing model data
        

        # Determine the oscillation
        if "nd" in r:
            x = 4
        elif "point_uncnfrm" in r:
            x = 3


        # Effectively oscillate between running every 4th and 3rd entry 
        if (counter == x): # If the counter is at the oscillating xth entry we're targeting
            print(r) # Print the layer name
            counter = 0 # Reset the count

            sum_fields = ps_records[i-2:i+1] # Select three fields of interest to sum
            sum_name = "iMap_"+r[13:] # Create desired field name
            express = "!"+sum_fields[0]+"!+!"+sum_fields[1]+"!+!"+sum_fields[2]+"!" # Create expression for calculating sum 
            arcpy.management.CalculateField(out_grid,sum_name,express) # Sum the fields
            keepFields.append(sum_name) # Add the summed field to the list of fields to keep when spatial joining
            for field in sum_fields: # Loop through the fields used when summing
                keepFields.remove(field) # Remove the field from the list of fields to keep
                arcpy.management.DeleteField(out_grid, field) # Delete the field from the feature since they're no longer necessary

        counter += 1 # Add 1 to the count

    # ----------


    input_grid = "tmp_grid_"+str(i) # Make the next run's target grid the output of this iteration's spatial join

    # If the last iteration, copy to a new feature
    if i == len(ps_records)-1:
        RAG_final = "SVI_Proj_reporting_analysis_grid_1km"
        arcpy.management.CopyFeatures(out_grid, RAG_final)

    print("All done")

NameError: name 'keepFields' is not defined

In [ ]:
# THIS CELL NEEDS TO BE EDITED TO REFLECT THE NEW METHOD OF CALCULATION

# Calculate per-species ND ratios, confirmed ratios, and unconfirmed ratios

for species in species_shortnames:
    print("Adding ratio fields")
    crName = "Cr_"+species
    cuName = "CUr_"+species
    ndName = "NDr_"+species

    arcpy.management.AddFields(RAG_final, [
        [crName, 'FLOAT'],
        [cuName, 'FLOAT'],
        [ndName, 'FLOAT']
    ])

    model_positives = "model_"+species
    iMap_cnfrm = "iMap_cnfrm_"+species
    iMap_uncnfrm = "iMap_uncnfrm_"+species
    iMap_nd = "iMap_nd_"+species

    # Whereclauses used when selecting records to calculate in the loop below
    whereClauses = {"model-only": "("+model_positives+" > 0 And ("+iMap_cnfrm+" = 0 or "+iMap_uncnfrm+" = 0)) or (model_points > 0 And "+iMap_nd+" = 0)", # These will be set to negative integers
                "iMap-only": "("+model_positives+" = 0 And ("+iMap_cnfrm+" > 0 or "+iMap_uncnfrm+" > 0)) or (model_points = 0 And "+iMap_nd+" > 0)", # These will be set to 0
                "Overlap": "("+model_positives+" > 0 And ("+iMap_cnfrm+" > 0 or "+iMap_uncnfrm+" > 0)) or (model_points > 0 And "+iMap_nd+" > 0)" # These will be set to positive floats
                # Cells with neither record type will retain a null designation during calculation
                }

    for i, clause in enumerate(whereClauses): # Loop to calculate the reporting analysis values
        print(whereClauses[clause])
        sel = arcpy.management.SelectLayerByAttribute(RAG_final, whereClauses[clause]) # Make the selection
        if i == 0: # Calculate model-only values
            for field in [crName, cuName]:
                arcpy.management.CalculateField(sel, field, "-!"+model_positives+"!") # Set the "ratio" as a negative to indicate only model data
            arcpy.management.CalculateField(sel, ndName, "-(!model_points! - !"+model_positives+"!)") # Calculate not-detected number
        elif i == 1: # Calculate iMap-only values
            for field in [crName, cuName, ndName]:
                arcpy.management.CalculateField(sel, field, "0") # Set the ratio as 0, since there's no model data here
        elif i == 2: # Calculate overlap values
            arcpy.management.CalculateField(sel, crName, "!"+model_positives+"!/!"+iMap_cnfrm+"!")
            arcpy.management.CalculateField(sel, cuName, "!model_"+model_positives+"!/(!"+iMap_cnfrm+"!+!"+iMap_uncnfrm+"!)")
            arcpy.management.CalculateField(sel, ndName, "(!model_points! - !"+model_positives+"!)/!"+iMap_nd+"!")
        else:
            print("Error: too many whereClauses")
        
        del sel
    del whereClauses

    print("Done!")

# The result of the loop above is the analysis-ready version (1 layer with numbers for all species)

# Build the viewing-ready version (n per-species layers combined into one)

# For each species:
    # Select records with non-null values for that species
    # Add & calculate common name, jurisdiction id, and op criteria fields
    # Add & calculate comparison percentile field (i.e., describes the percentile of model-only record magnitude or imap-model overlap ratio)
# Merge layers into one for upload into AGOL